### Imports

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [13]:
# Carregar dados
file_path = 'C:\\Users\\Yuki\\Desktop\\github\\Aula_DS\\brasileirao_predicition\\data.csv'
data = pd.read_csv(file_path)

In [14]:
# Ordenar os dados por data para garantir a sequência correta nos cálculos acumulados
data.sort_values(by=['ano_campeonato', 'data'], inplace=True)

# Calcular médias acumuladas e diferença de gols para cada time
data['gols_marcados_mandante'] = data.groupby('time_mandante')['gols_mandante'].expanding().mean().shift().reset_index(level=0, drop=True)
data['gols_sofridos_mandante'] = data.groupby('time_mandante')['gols_visitante'].expanding().mean().shift().reset_index(level=0, drop=True)
data['gols_marcados_visitante'] = data.groupby('time_visitante')['gols_visitante'].expanding().mean().shift().reset_index(level=0, drop=True)
data['gols_sofridos_visitante'] = data.groupby('time_visitante')['gols_mandante'].expanding().mean().shift().reset_index(level=0, drop=True)
data['diferenca_gols_mandante'] = data.groupby('time_mandante')['gols_mandante'].expanding().sum().shift().reset_index(level=0, drop=True) \
                                 - data.groupby('time_mandante')['gols_visitante'].expanding().sum().shift().reset_index(level=0, drop=True)
data['diferenca_gols_visitante'] = data.groupby('time_visitante')['gols_visitante'].expanding().sum().shift().reset_index(level=0, drop=True) \
                                  - data.groupby('time_visitante')['gols_mandante'].expanding().sum().shift().reset_index(level=0, drop=True)

In [15]:
# Supondo a existência das colunas 'gols_mandante' e 'gols_visitante'
data['resultado'] = (data['gols_mandante'] > data['gols_visitante']).astype(int) - \
                    (data['gols_mandante'] < data['gols_visitante']).astype(int)

# Selecionando características relevantes
# Lista de features para o modelo incluindo novas features
features = ['ano_campeonato', 'rodada', 'time_mandante', 'time_visitante', 'publico',
            'gols_marcados_mandante', 'gols_sofridos_mandante', 'gols_marcados_visitante',
            'gols_sofridos_visitante', 'diferenca_gols_mandante', 'diferenca_gols_visitante']

data_filtered = data[features + ['resultado']].dropna()

# Convertendo times para categorias numéricas
data_filtered['time_mandante'] = data_filtered['time_mandante'].astype('category').cat.codes
data_filtered['time_visitante'] = data_filtered['time_visitante'].astype('category').cat.codes

# Dividindo os dados
train_data, remaining_data = train_test_split(data_filtered, train_size=0.65, random_state=42)
validation_size = 0.15 / (1 - 0.65)  # Proporção do que sobrou após a divisão de treino
validation_data, test_data = train_test_split(remaining_data, train_size=validation_size, random_state=42)

# Preparando o target para o XGBoost
le = LabelEncoder()
y_train = le.fit_transform(train_data['resultado'])
y_validation = le.transform(validation_data['resultado'])

# Definindo o modelo XGBoost
xgb_model = XGBClassifier(eval_metric='mlogloss')

# Grid de parâmetros para o GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Configurando o GridSearchCV apenas uma vez
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1
)

# Preparando dados de treino e validação
X_train = train_data.drop('resultado', axis=1)
y_train = le.fit_transform(train_data['resultado'])
X_validation = validation_data.drop('resultado', axis=1)
y_validation = le.transform(validation_data['resultado'])

# Treinamento com GridSearchCV
grid_search.fit(X_train, y_train)

# Melhor modelo e validação
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_validation)
validation_accuracy = accuracy_score(y_validation, y_pred)

print("Melhores parâmetros:", grid_search.best_params_)
print("Acurácia na validação:", validation_accuracy)

Fitting 5 folds for each of 2916 candidates, totalling 14580 fits
Melhores parâmetros: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 10, 'n_estimators': 100, 'subsample': 0.8}
Acurácia na validação: 0.5119047619047619
